In [1]:
import pandas as pd 

In [39]:
def fetch_results(results_file):

    df = pd.read_csv(results_file).drop(columns = ["Unnamed: 0"])
    return df


lstm_results_file = "Saved_models\\LSTM_models_1st_iter\\Model_96\\y_test_y_pred.csv"
temp = fetch_results(lstm_results_file)

In [29]:
cnn_results_file = "Saved_models\\CNN_models_1st_iter\\\Model_87\\y_test_y_pred.csv"
fetch_results(cnn_results_file)

,y_test,y_pred
0,-5.1,-0.295970
1,-4.1,-0.134379
2,-7.0,-0.126345
3,-5.8,0.029337
4,-4.9,0.221473
...,...,...
35849,-5.3,-0.516042
35850,-4.6,-0.374934
35851,0.0,-0.411533
35852,0.0,-0.307701


In [59]:
def discard_indeces_pred(df, bound, keep_middle = True):
    for idx,val in enumerate(df['y_pred'].copy()):
        if keep_middle:
            if (-bound < val < bound):
                pass
            else: 
                df.drop(idx, inplace = True)
                
        else: 
            if bound > abs(val):
                df.drop(idx, inplace = True)
                            
    return df.reset_index(drop=True)

In [67]:
def calc_pip_ratio(df):
    error = 0
    y_test,y_pred = df['y_test'] ,df['y_pred']
    
    len_df = len(df)
    if len_df == 0:
        len_df = 1
    
    abs_sum_pips = sum([abs(y_test[i]) for i in range(len(df))])
    
    # y_pred defines the direction of the trade. Negative pips mean a possitive trade, if shorted.
    sum_pips = sum([y_test[i] if y_pred[i] >= 0 else -y_test[i] for i in range(len(df))]) 
    pip_rat =  sum_pips / abs_sum_pips
    
    return pip_rat, len_df
       
df = fetch_results(base_results_file)  
pip_rat,  len_df = calc_pip_volatility(df)

pip_rat,  len_df

(0.008643635285113858, 37377)

In [69]:
for i in range(4,41,4):
    lstm_pip_rat,  len_df = calc_pip_ratio(discard_indeces_pred(fetch_results(lstm_results_file),i, keep_middle = False))
    print(f"LSTM pip ratio of predictions with absolute value greater than {i}: ", round(lstm_pip_rat,3), f"  samples: {len_df}")

LSTM pip ratio of predictions with absolute value greater than 4:  0.05   samples: 20971
LSTM pip ratio of predictions with absolute value greater than 8:  0.075   samples: 11327
LSTM pip ratio of predictions with absolute value greater than 12:  0.095   samples: 6083
LSTM pip ratio of predictions with absolute value greater than 16:  0.14   samples: 3386
LSTM pip ratio of predictions with absolute value greater than 20:  0.209   samples: 1908
LSTM pip ratio of predictions with absolute value greater than 24:  0.25   samples: 1087
LSTM pip ratio of predictions with absolute value greater than 28:  0.326   samples: 596
LSTM pip ratio of predictions with absolute value greater than 32:  0.489   samples: 298
LSTM pip ratio of predictions with absolute value greater than 36:  0.528   samples: 159
LSTM pip ratio of predictions with absolute value greater than 40:  0.75   samples: 89


In [79]:
for i in range(3,31,3):
    i = round(i/10,3)
    cnn_pip_rat,  len_df = calc_pip_ratio(discard_indeces_pred(fetch_results(cnn_results_file),i, keep_middle = False))
    print(f"CNN pip ratio of predictions with absolute value greater than {i:.4f}: ", round(cnn_pip_rat,3), f"  samples: {len_df}")

CNN pip ratio of predictions with absolute value greater than 0.3000:  0.017   samples: 19250
CNN pip ratio of predictions with absolute value greater than 0.6000:  0.031   samples: 9244
CNN pip ratio of predictions with absolute value greater than 0.9000:  0.081   samples: 4281
CNN pip ratio of predictions with absolute value greater than 1.2000:  0.132   samples: 2200
CNN pip ratio of predictions with absolute value greater than 1.5000:  0.103   samples: 1142
CNN pip ratio of predictions with absolute value greater than 1.8000:  0.134   samples: 580
CNN pip ratio of predictions with absolute value greater than 2.1000:  0.211   samples: 315
CNN pip ratio of predictions with absolute value greater than 2.4000:  0.296   samples: 174
CNN pip ratio of predictions with absolute value greater than 2.7000:  0.336   samples: 103
CNN pip ratio of predictions with absolute value greater than 3.0000:  0.364   samples: 57


$$
Pips\ Gained = \sum_{i=1}^{n}{\left\{\begin{matrix} 
|y_i| ,& sgn(y_i) = sgn(\hat{y}_i) \\ 
-|y_i| ,& sgn(y_i) \neq sgn(\hat{y}_i) \\
\end{matrix}\right.} \\
\\
Pip\ ratio = \frac{Pips\ Gained}{\sum_{i=1}^{n}{|y_i|}}
$$
